In [22]:
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt, animation
import proplot as plot
import seaborn as sns

sys.path.append('/Users/46h/Research/code/accphys/')
from tools import plotting as myplt
from tools import animation as myanim
from tools import utils

sys.path.append('/Users/46h/Research/code/accphys/pyorbit_studies/measurement/')
from data_analysis import to_vec, to_mat, reconstruct_moments

In [25]:
plot.rc['figure.facecolor'] = 'white'
plot.rc['cycle'] = 'default'
plot.rc['savefig.dpi'] = 'figure'
plot.rc['animation.html'] = 'jshtml'

## Load data 

In [ ]:
Sigma0 = 1e6 * np.loadtxt('_output/data/Sigma0.dat')
X0 = 1e3 * np.loadtxt('_output/data/X0.dat')

In [43]:
def load(handle, ws_name):
    path = '_output/data/{}_{}.npy'.format(handle, ws_name)
    return np.load(path)

phases_dict = {ws: load('phases', ws) for ws in ws_names}
moments_dict = {ws: 1e6 * load('moments', ws) for ws in ws_names}
transfer_mats_dict = {ws: load('transfer_mats', ws) for ws in ws_names}
env_params_dict = {ws: 1e3 * load('env_params', ws) for ws in ws_names}
coords_dict_dict = {ws: 1e3 * load('X', ws) for ws in ws_names}

phases = np.vstack([phases_dict[ws] for ws in active_ws_names])
moments = np.vstack([moments_dict[ws] for ws in active_ws_names]) # measured 
transfer_mats = np.vstack([transfer_mats_dict[ws] for ws in active_ws_names])
env_params = np.vstack([env_params_dict[ws] for ws in active_ws_names])
coords = np.vstack([coords_dict_dict[ws] for ws in active_ws_names])
    
Sigma0 = 1e6 * np.loadtxt('_output/data/Sigma0.dat')
X0 = 1e3 * np.loadtxt('_output/data/X0.dat')

## Reconstruction

In [11]:
def reconstruct(transfer_mats, moments):
    A, b = [], []
    for M, (x2, y2, xy) in zip(transfer_mats, moments):
        A.append([M[0, 0]**2, M[0, 1]**2, 2*M[0, 0]*M[0, 1], 0, 0, 0, 0, 0, 0, 0])
        A.append([0, 0, 0, M[2, 2]**2, M[2, 3]**2, 2*M[2, 2]*M[2, 3], 0, 0, 0, 0])
        A.append([0, 0, 0, 0, 0, 0, M[0, 0]*M[2, 2],  M[0, 1]*M[2, 2],  M[0, 0]*M[2, 3],  M[0, 1]*M[2, 3]])
        b.append(x2)
        b.append(y2)
        b.append(xy)
    A, b = np.array(A), np.array(b)
    Sigma = reconstruct_moments(A, b)
    return Sigma